## ECE-UY 4563 Intro to Machine Learning Final Project - Dog Breed Classification 
* Presented by: Eren Tuksal(et2179@nyu.edu) and James Zhang(hz2852@nyu.edu)

In this project, we aim to develop a convolutional neural network capable of identifying 120 different dog breeds. The primary training dataset includes over 10,000 images of dogs captured from various angles and in diverse scenes. Additionally, there is a test dataset that includes not only dog images but also images of non-dog subjects, allowing us to evaluate the model's ability to differentiate between dog and non-dog images.

The following topics and techniques are covered:
* PyTorch
* ..


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, preprocessing

df = pd.read_csv('labels.csv')

(10222, 2)

Part 1 - Check data
Part 2 - Input images
Part 3 - Pick a model